In [ ]:
#Dependencies
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import requests

Rafael

In [ ]:
#Getting country codes
countrycodes_file = "Resources/CountryCodesLookup.csv"
countrycodes = pd.read_csv(countrycodes_file, encoding="ISO-8859-1")
countrycodes = countrycodes.rename(columns={'COUNTRY':'country'})

#Getting happiness index
happiness_file = "Resources/WorldHappinessReport.csv"
happiness = pd.read_csv(happiness_file, encoding="ISO-8859-1")

happiness2017 = happiness.loc[happiness['year']==2017,['country','Life Ladder']]
print(happiness2017.head())

#Merging both
mymerge = pd.merge(countrycodes, happiness2017, how="left", on=['country'])
mymerge = mymerge.drop(['A3 (UN)','NUM (UN)','DIALING CODE','spotifyFlag'],axis=1)
print(mymerge.head())

In [ ]:
#Checking indicators and grabbing interesting data for analysis
baseurl = "http://api.worldbank.org/v2/country/"
year = "2017"

def grabindicator(y):
    if y == 0: #Surface area (sq.km)
        x = "AG.SRF.TOTL.K2"
    elif y == 1: #Population, total
        x = "SP.POP.TOTL"
    elif y == 2: # Labor Force
        x = "SL.TLF.TOTL.IN"
    elif y == 3: # Unemployment, total (% of total labor force) (modeled ILO estimate)
        x = "SL.UEM.TOTL.ZS"
    else:
        print(f'Indicator "{y}" not available')
    return x

temp = []
surfacearea = []
population = []
laborforce = []
unemployment = []

countryfound = []

def lists(indicatorcod, x, lock, errorfree):

    try:   
        if indicatorcod == 0: #Surface area (sq.km)
            temp = x[1]
            temp = temp[0]
            surfacearea.append(temp['value'])
    
        elif indicatorcod == 1: #Population, total
            temp = x[1]
            temp = temp[0]
            population.append(temp['value'])         

        elif indicatorcod == 2: # Labor Force
            temp = x[1]
            temp = temp[0]
            laborforce.append(temp['value'])

        elif indicatorcod == 3: # Unemployment, total (% of total labor force) (modeled ILO estimate)
            temp = x[1]
            temp = temp[0]
            print (temp)
            print(temp['value'])
            unemployment.append(temp['value'])       
        
        else:
            print("*** Error on function 'lists' ***")

        
        if lock != 1 and errorfree == 1 and indicatorcod <1:
                countryfound.append(pais)
                lock = 1
        
    except TypeError:
        print(f'Error: Country Code "{countrycode}" not found')



for i in range (4):
    indicatorcode = grabindicator(i)
    indicatorcounter = int(i)
    lock = 0

    for code,pais in zip(mymerge['A2 (ISO)'], mymerge['country']):        
        countrycode = code
        print(pais)        
        print(indicatorcounter)
        query_url = f"{baseurl}{countrycode}/indicator/{indicatorcode}?date={year}&format=json"
        print (query_url)
        response = requests.get(query_url).json()
        print(f'response is {response}\n')
        
        if len(response) < 2:
            print(f'response for {countrycode} is invalid\n')
            errorfree = 0
        elif response[1] is None:
            print(f'response for {countrycode} is None \n')
            errorfree = 0
        else:
            errorfree = 1
            lists(indicatorcounter, response, lock, errorfree)
                                  

print("\n***********all data was collected via the API")

countryfound1 = pd.DataFrame(countryfound)


#Compiling grabbed data from API Calls into one data frame
mymerge_dict = {
    "Country": countryfound,
    "Surface Area": surfacearea,
    "Total Population": population,
    "Labor Force": laborforce,
    "Total Unemployment": unemployment}
mymerge_data = pd.DataFrame(mymerge_dict)

mymerge_data = pd.merge(mymerge, mymerge_data, how="left", left_on=['country'],right_on=['Country'])
mymerge_data = mymerge_data.drop(['Country'],axis=1)

#Saving all data to disk/file
outputfileCSV = open('Outputs/TotalUnemploymentVsHappinessPlot-Rafael.csv', 'w+' )
outputfile1 = mymerge_data.to_csv(index=True)
outputfileCSV.write(outputfile1)
outputfileCSV.close()

In [ ]:
#Plotting
y_axis1 = mymerge_data['Life Ladder']       #use this for all plots
x_axis1 = mymerge_data['Total Unemployment']

Unemployment, ax = plt.subplots()
Unemployment = plt.scatter(x_axis1,y_axis1,marker='s', alpha=0.75, color = "yellow",edgecolor ="black", label = '% Total Unemployment')

# Set line for regression 
##(slope, intercept, _, _, _) = sci.stats.linregress(x_axis1, y_axis1)
##fit = slope * x_axis1 + intercept

# Plot line
##ax.plot(x_axis1, fit, 'b--')


ax.legend(loc="best")
ax.legend()
ax.grid(True)
plt.title("Unemployment vs Happiness")
plt.ylabel("Happiness index")
plt.xlabel("Unemployment")

#Save plot to file
Unemployment,plt.savefig("Graphs/TotalUnemploymentVsHappinessPlot-RafaelSantos.png")

Arghavan

In [ ]:
#Cleaning Happiness DF
Happiness =  pd.read_csv('Resources/WorldHappinessReport.csv')
Happiness_New = Happiness.drop(columns=['Delivery Quality', 'Democratic Quality', 
                                        'Standard deviation of ladder by country-year',
                                        'Standard deviation/Mean of ladder by country-year',
                                        'GINI index (World Bank estimate)', 'GINI index (World Bank estimate), average 2000-15',
                                        'gini of household income reported in Gallup, by wp5-year'])

Happiness_Updated = Happiness_New.loc[Happiness_New["year"]==2016,:]
Happiness_Olympics = Happiness_Updated.dropna()

In [ ]:
#Cleaning Olympics DF
CSV_athletes = pd.read_csv("Resources/120-years-of-olympic-history-athletes-and-results/athlete_events.csv")
CSV_countries =  pd.read_csv("Resources/120-years-of-olympic-history-athletes-and-results/noc_regions.csv")

#Cleaning Athletes DF
CSV_athletes = CSV_athletes.drop(columns=['Season', 'City', 'Height', 'Weight'])
CSV_athletes = CSV_athletes.loc[CSV_athletes["Year"]==2016,:]
CSV_athletes_updated = CSV_athletes.fillna("none")

#Merging Athletes and Country DF
merged_athletes = pd.merge(CSV_athletes_updated, CSV_countries, on = "NOC", how = "left")
merged_athletes = merged_athletes.rename(columns= {"region" : "country"})

#Getting Number of Atheletes
merged_athletes1 = merged_athletes.groupby(merged_athletes["country"])
Final_athletes = merged_athletes1["Name"].count()
Final_df = pd.DataFrame(Final_athletes)
Final_df = Final_df.rename(columns= {"Name" : "Number of Athletes"})

# Changin USA to United States so it merges with Happiness DF
Final_df = Final_df.rename(index={"USA":'United States'})

In [ ]:
#Creating Final DF
Athlete_Happiness = pd.merge(Final_df, Happiness_Olympics, on = "country")
Athlete_Happiness.to_csv("Outputs/Athlete_Happiness.csv")

Shayan

In [ ]:
happiness_df = pd.read_csv("Resources/WorldHappinessReport.csv")
tourism_df = pd.read_csv("Resources/Tourism.csv", skiprows=3)
happiness17_df = happiness_df.loc[(happiness_df["year"] == 2017)]
happiness_score = happiness17_df[["country", "Life Ladder"]]
happiness_score.columns = ["Country Name", "Happines Score"]
tourism_number = tourism_df[["Country Name", "2017"]]
tourism_number = tourism_number.fillna(0)
merge_table = pd.merge(happiness_score, tourism_number, on = "Country Name")
merge_table.to_csv("Outputs/Tourism_Happiness.csv")


Mariana

Sandra